# Домашнее задание № 3. Исправление опечаток

## 1. Доп. ранжирование по вероятности (3 балла)

Дополните get_closest_hybrid_match в семинаре так, чтобы из кандадатов с одинаковым расстоянием редактирования выбиралось наиболее вероятное.

In [120]:
import os, regex, re
from string import punctuation
import numpy as np
import json
from collections import Counter
from pprint import pprint
from nltk import sent_tokenize
punctuation += "«»—…“”"
punct = set(punctuation)
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
import numpy as np
from collections import Counter
from tqdm import tqdm
import textdistance
from operator import itemgetter

In [2]:
bad = open('sents_with_mistakes.txt', encoding='utf8').read().splitlines()
true = open('correct_sents.txt', encoding='utf8').read().splitlines()

In [3]:
# напишем функцию, которая будет сопоставлять слова в правильном и ошибочном варианте
# разобьем предложение по пробелам и удалим пунктуация на границах слов
def align_words(sent_1, sent_2):
    tokens_1 = sent_1.lower().split()
    tokens_2 = sent_2.lower().split()
    
    tokens_1 = [token.strip(punctuation) for token in tokens_1]
    tokens_2 = [token.strip(punctuation) for token in tokens_2]
    
    tokens_1 = [token for token in tokens_1 if token]
    tokens_2 = [token for token in tokens_2 if token]
    
    assert len(tokens_1) == len(tokens_2)
    
    return list(zip(tokens_1, tokens_2))

In [8]:
mistakes = []
total = 0
for i in range(len(true)):
    word_pairs = align_words(true[i], bad[i])
    
    
    for pair in word_pairs:
        if pair[0] != pair[1]:
            mistakes.append(pair)
        total += 1

In [9]:
print('Доля ошибок - ', len(mistakes)/total )

Доля ошибок -  0.12886443221610805


In [13]:
corpus = open('wiki_data.txt', encoding='utf8').read()

In [19]:
vocab = Counter(re.findall('\w+', corpus.lower()))

word2id = list(vocab.keys())
id2word = {i:word for i, word in enumerate(vocab)}


vec = CountVectorizer(analyzer='char', ngram_range=(1,1))
X = vec.fit_transform(vocab)

In [145]:
def get_closest_match_with_metric(text, lookup,topn=20, metric=textdistance.levenshtein):
    
    similarities = Counter()
    
    for word in lookup:
        similarities[word] = metric.normalized_similarity(text, word) 
    
    return similarities.most_common(topn)

def get_closest_match_vec(text, X, vec, topn=20):
    v = vec.transform([text])
    
    similarities = cosine_distances(v, X)[0]
    topn = similarities.argsort()[:topn] 
    
    return [(id2word[top], similarities[top]) for top in topn]

N = sum(vocab.values())

def P(word, N=N):
    return np.log(vocab[word] / N)

def predict_mistaken(word, vocab):
    return 0 if word in vocab else 1

def get_closest_hybrid_match(text, X, vec, topn=3, metric=textdistance.damerau_levenshtein):
    candidates = get_closest_match_vec(text, X, vec, topn*8)
    lookup = [cand[0] for cand in candidates]
    #print(lookup)
    
    closest = get_closest_match_with_metric(text, lookup, topn * 3, metric=metric)
    #print(closest)
    
    min_d = max(closest, key=itemgetter(1))[1]

    max_p = -np.inf
    
    for word, dist in closest:
        if P(word) > max_p:
            target = (word, dist, P(word))
            max_p = P(word)

        if dist < min_d:
            break
    
    return target

Стоит заметить, что у подхода с CountVectorizer тоже есть некоторые недостатки - как минимум, слова с имеющимися в слове буквами в любом случае будут получать значительно более близкие косинусные расстояния, т.к. слова с отсутствующими буквами будут ортогональными как минимум в одной из размерностей. Таким образом смещая ожидаемые вероятности при финальном выборе - так, в примере ниже, чтобы получить слово выдра необходимо ровно столько же операций, как и для слово __сыра__, но оно уже не прошло, отфильтровавшись ранее.

In [51]:
get_closest_hybrid_match("сфыдра", X, vec)

['фарсы', 'сарды', 'сафры', 'садыр', 'фасады', 'арсыфа', 'рассады', 'сандры', 'русафы', 'сыра', 'арфы', 'сары', 'сфар', 'сафы', 'расф', 'рады', 'дыра', 'садр', 'дарс', 'дары', 'фарс', 'фары', 'сады', 'расы']
[('сыра', 0.6666666666666667), ('садыр', 0.5), ('сандры', 0.5), ('сфар', 0.5), ('дыра', 0.5), ('садр', 0.5), ('сарды', 0.33333333333333337), ('сафры', 0.33333333333333337), ('сары', 0.33333333333333337)]


('сыра', 0.6666666666666667, 3.1036147607142114e-06)

In [54]:
mistakes = []
total_mistaken = 0
mistaken_fixed = 0

total_correct = 0
correct_broken = 0

total = 0
correct = 0

cashed = {}
for i in tqdm(range(len(true))):
    word_pairs = align_words(true[i], bad[i])
    for pair in word_pairs:
        if predict_mistaken(pair[1], vocab):
            pred = cashed.get(pair[1], get_closest_hybrid_match(pair[1], X, vec)[0][0])
            cashed[pair[1]] = pred
        else:
            pred = pair[1]
        
            
        if pred == pair[0]:
            correct += 1
        else:
            mistakes.append((pair[0], pair[1], pred))
        total += 1
            
        if pair[0] == pair[1]:
            total_correct += 1
            if pair[0] != pred:
                correct_broken += 1
        else:
            total_mistaken += 1
            if pair[0] == pred:
                mistaken_fixed += 1


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 915/915 [04:01<00:00,  3.79it/s]


Это предсказуемо не решило проблему с появлением новых ошибок, но могло сделать получающиеся в результате коррекции слова чуть более ожидаемыми.

In [56]:
print(f"Accuracy: {correct/total}")
print(f"Mistakes not fixed: {mistaken_fixed/total_mistaken}")
print(f"Mistakes made: {correct_broken/total_correct}")

Accuracy: 0.7928964482241121
Mistakes not fixed: 0.0015527950310559005
Mistakes made: 0.09004249454461927


## 2.  Symspell (7 баллов)

А еще, кажется, в силу наличия ударений в википедии, наша регулярка сработала слегка против нас, добавив ошибок в словарь. Попробуем пофиксить. Хотя в таком случае в нашем словаре одно и то же слово с ударением или без будут представлены разными входами.

In [61]:
corpus[:100]

'######Новостройка (Нижегородская область)############Новостро́йка — сельский посёлок в Дивеевском ра'

In [81]:
re.findall('\w+', corpus.lower())[3]

'новостро'

### "Эксперименты"

In [76]:
char = "о́"
char2 = "о"
print(char.encode())
print(char2.encode())

b'\xd0\xbe\xcc\x81'
b'\xd0\xbe'


In [103]:
char_t = b"\xcc\x81"

regex = "[\w]+"

In [149]:
regex.findall(r"[\p{L}\p{M}\d'-]+", corpus.lower())[3]

'новостро́йка'

### Новый словарь

In [144]:
vocab = Counter(regex.findall(r"[\p{L}\p{M}\d'-]+", corpus.lower()))

word2id = list(vocab.keys())
id2word = {i:word for i, word in enumerate(vocab)}


vec = CountVectorizer(analyzer='char', ngram_range=(1,1))
X = vec.fit_transform(vocab)

In [146]:
mistakes = []
total_mistaken = 0
mistaken_fixed = 0

total_correct = 0
correct_broken = 0

total = 0
correct = 0

cashed = {}
for i in tqdm(range(len(true))):
    word_pairs = align_words(true[i], bad[i])
    for pair in word_pairs:
        if predict_mistaken(pair[1], vocab):
            pred = cashed.get(pair[1], get_closest_hybrid_match(pair[1], X, vec)[0][0])
            cashed[pair[1]] = pred
        else:
            pred = pair[1]
        
            
        if pred == pair[0]:
            correct += 1
        else:
            mistakes.append((pair[0], pair[1], pred))
        total += 1
            
        if pair[0] == pair[1]:
            total_correct += 1
            if pair[0] != pred:
                correct_broken += 1
        else:
            total_mistaken += 1
            if pair[0] == pred:
                mistaken_fixed += 1


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 915/915 [03:57<00:00,  3.85it/s]


Ну, чуть лучше.

In [147]:
print(f"Accuracy: {correct/total}")
print(f"Mistakes not fixed: {mistaken_fixed/total_mistaken}")
print(f"Mistakes made: {correct_broken/total_correct}")

Accuracy: 0.7987993996998499
Mistakes not fixed: 0.0015527950310559005
Mistakes made: 0.08326633742965431


Реализуйте алгоритм Symspell. Он похож на алгоритм Норвига, но проще и быстрее. Он основан только на одной операции - удалении символа. Описание алгоритма по шагам:

1) Составляется словарь правильных слов  
2) На основе словаря правильных слов составляется словарь удалений - для каждого правильного слова создаются все варианты удалений и создается словарь, где ключ - слово с удалением, а значение - правильное слово  (!) 
3) Для выбора исправления для слова с опечаткой генерируются все варианты удаления, из них выбираются те, что есть в словаре удалений, построенного на шаге 2. Слово с опечаткой заменяется на правильное слово, соответствующее варианту удаления  
4) Если в словаре удалений есть несколько вариантов, то выбирается удаление, которому соответствует наиболее вероятное правильное слово  


Оцените качество полученного алгоритма теми же тремя метриками.

Вообще алгоритм явно позволяет исправлять на нужные написания только случаи опечаток, что в общем случае верно далеко не всегда...

In [266]:
def get_del_dict(vocab: Counter, deletions = 1):

    del_dict = {}
    
    for word in vocab:
        for j in range(deletions):
            for i in range(len(word)):

                del_dict[word[:i] + word[i+j:]] = word

    return del_dict            

In [267]:
N = sum(vocab.values())

In [268]:
deletions = 2

In [269]:
lookup_dict = get_del_dict(vocab, deletions=deletions)

Для удобства будем таки брать логарифм при расчете вероятностей

In [270]:
def get_closest_symspell(word: str, del_dict: dict, vocab: dict, deletions: int, N: int):

    def P(word, N=N):
        return np.log(vocab[word] / N)

    vars = []
    finishers = []

    for j in range(deletions):
        for i in range(len(word)):
            vars.append(word[:i] + word[i+j:])
    
    for str in vars:
        if str in del_dict:
            finishers.append(del_dict[str])
    try:
        return max(finishers, key=P)
    except ValueError:
        return word

In [271]:
mistakes = []
total_mistaken = 0
mistaken_fixed = 0

total_correct = 0
correct_broken = 0

total = 0
correct = 0

cashed = {}
for i in tqdm(range(len(true))):
    word_pairs = align_words(true[i], bad[i])
    for pair in word_pairs:
        if predict_mistaken(pair[1], vocab):
            pred = cashed.get(pair[1], get_closest_symspell(pair[1], lookup_dict, vocab, deletions, N))
            cashed[pair[1]] = pred
        else:
            pred = pair[1]
        
            
        if pred == pair[0]:
            correct += 1
        else:
            mistakes.append((pair[0], pair[1], pred))
        total += 1
            
        if pair[0] == pair[1]:
            total_correct += 1
            if pair[0] != pred:
                correct_broken += 1
        else:
            total_mistaken += 1
            if pair[0] == pred:
                mistaken_fixed += 1


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 915/915 [00:00<00:00, 21662.23it/s]


Между тем, результат заметно лучше.

In [272]:
print(f"Accuracy: {correct/total}")
print(f"Mistakes not fixed: {mistaken_fixed/total_mistaken}")
print(f"Mistakes made: {correct_broken/total_correct}")

Accuracy: 0.8784392196098049
Mistakes not fixed: 0.40683229813664595
Mistakes made: 0.051797404387274606
